In [4]:
import pandas as pd


In [338]:
def basic_estimator(pred_class_list, correct_class_list, no_harvest):
    total_classes = 10
    total_cost = 0
    total_data_count = len(pred_class_list)
    valid_data_count = 0
    for i in range(total_data_count):
        pred_class = pred_class_list[i]
        correct_class = correct_class_list[i]
        if pred_class > correct_class or pred_class == 10 or no_harvest:
            if pred_class<correct_class:
                cost = correct_class-pred_class+total_classes
            else:
                cost = pred_class-correct_class
            total_cost += cost
            valid_data_count += 1
            #print "[pred,target,cost]: [{},{},{}]".format(pred_class, correct_class, cost)
    print "total valid data/total data: {}/{}".format(valid_data_count, total_data_count)
    avg_cost = 1.0*total_cost/total_data_count
    print "sum/count = {}/{}={}".format(total_cost, total_data_count, avg_cost)


In [299]:
print "===Deploy M_full==="
dir_name = "results-0522T0733-learning-1-5ms-deploy-full"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = False
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy M_full===

use pred_class_list:
total valid data/total data: 9408/11856
sum/count = 12619/11856=1.06435560054

use upper_bound_list:
total valid data/total data: 9423/11856
sum/count = 12654/11856=1.06730769231


In [300]:
print "===Deploy M_partial==="
dir_name = "results-0522T0737-learning-1-5ms-deploy-partial"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = False
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy M_partial===

use pred_class_list:
total valid data/total data: 204/11896
sum/count = 204/11896=0.0171486213853

use upper_bound_list:
total valid data/total data: 349/11896
sum/count = 352/11896=0.0295897780767


In [339]:
print "===Deploy shadow M_partial==="
dir_name = "results-0524T1645-learning-1-5ms-no-harvest-deploy-partial-no-pred-max-capped"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = True
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy shadow M_partial===

use pred_class_list:
total valid data/total data: 0/11954
sum/count = 0/11954=0.0

use upper_bound_list:
total valid data/total data: 38/11954
sum/count = 40/11954=0.00334616028108


In [302]:
print "===Deploy shadow M_partial==="
dir_name = "results-0524T1645-learning-1-5ms-no-harvest-deploy-partial-no-pred-max-capped"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = []
pred_class_list = []
upper_bound_list = []
for i in range(len(df.iteration)-1):
    correct_class = df.cpu_max[i+1]
    pred_class = df.pred_peak[i]
    logged_upper_bound = df.upper_bound[i]
    if i==0:
        current_busy_cpu = df.primary_busy_cores[i]
    else:
        current_busy_cpu = min(df.primary_busy_cores[i], upper_bound_list[i-1])
    upper_bound = max(pred, current_busy_cpu+1)
    upper_bound_list.append(upper_bound)
    correct_class_list.append(correct_class)
    pred_class_list.append(pred_class)
    if logged_upper_bound != upper_bound:
        print "logged_upper_bound {} != upper_bound {}".format(logged_upper_bound, upper_bound)

no_harvest = True
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy shadow M_partial===
logged_upper_bound 10 != upper_bound 4

use pred_class_list:
total valid data/total data: 11954/11954
sum/count = 148256/11954=12.4022084658

use upper_bound_list:
total valid data/total data: 11954/11954
sum/count = 134822/11954=11.2784005354


In [336]:
print "===Deploy shadow M_partial==="
dir_name = "results-0524T1723-learning-1-5ms-no-harvest-deploy-partial-no-pred-max-capped-3iter"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = True
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy shadow M_partial===

use pred_class_list:
total valid data/total data: 11950/11950
sum/count = 147840/11950=12.3715481172

use upper_bound_list:
total valid data/total data: 11950/11950
sum/count = 134589/11950=11.2626778243


In [321]:
print "===Deploy M_full (+explore)==="
dir_name = "results-0524T1723-learning-1-5ms-no-harvest-deploy-partial-no-pred-max-capped-3iter"

for i in range(3):
    print "-------- iter {} --------".format(i)
    log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/'+str(i)+'/hvmagent.csv'
    df = pd.read_csv(log_path)

    correct_class_list = df.cpu_max[1:].reset_index(drop=True)
    pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
    upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

    no_harvest = True
    print "\nuse pred_class_list:"
    avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

    print "\nuse upper_bound_list:"
    avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)


===Deploy M_full (+explore)===
-------- iter 0 --------

use pred_class_list:
total valid data/total data: 11950/11950
sum/count = 147840/11950=12.3715481172

use upper_bound_list:
total valid data/total data: 11950/11950
sum/count = 134589/11950=11.2626778243
-------- iter 1 --------

use pred_class_list:
total valid data/total data: 11946/11946
sum/count = 148145/11946=12.4012221664

use upper_bound_list:
total valid data/total data: 11946/11946
sum/count = 134988/11946=11.2998493219
-------- iter 2 --------

use pred_class_list:
total valid data/total data: 11948/11948
sum/count = 148106/11948=12.395882156

use upper_bound_list:
total valid data/total data: 11948/11948
sum/count = 134508/11948=11.2577837295


In [289]:
print "===Deploy M_partial (+explore)==="
dir_name = "results-0522T0822-learning-1-5ms-deploy-partial-explore"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = False
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy M_partial (+explore)===

use pred_class_list:
total valid data/total data: 2730/11845
sum/count = 8605/11845=0.726466863656

use upper_bound_list:
total valid data/total data: 3432/11845
sum/count = 9364/11845=0.790544533558


In [290]:
print "===Deploy shadow M_partial (+explore)==="
dir_name = "results-0522T0824-learning-1-5ms-no-harvest-deploy-partial-no-pred-max-capped-explore"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path)

correct_class_list = df.cpu_max[1:].reset_index(drop=True)
pred_class_list = df.pred_peak[:-1].reset_index(drop=True)
upper_bound_list = df.upper_bound[:-1].reset_index(drop=True)

no_harvest = True
print "\nuse pred_class_list:"
avg_cost_eval_actual_run = basic_estimator(pred_class_list, correct_class_list, no_harvest)

print "\nuse upper_bound_list:"
avg_cost_eval_actual_run = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

===Deploy shadow M_partial (+explore)===

use pred_class_list:
total valid data/total data: 11955/11955
sum/count = 56874/11955=4.75734002509

use upper_bound_list:
total valid data/total data: 11955/11955
sum/count = 8533/11955=0.713759933082


In [84]:
print "===Evaluating M_partial on biased data ==="
no_harvest = True
avg_cost_eval_actual_run = basic_estimator(pred_class_lines, correct_class_lines, no_harvest)

===Evaluating M_partial on biased data ===
total valid data/total data: 9408/9408
avg cost: 11.9345238095


In [323]:
# extract biased data from explore
dir_name = "results-0524T1732-learning-1-5ms-deploy-full-explore-3iter"
for index in range(3):
    log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/'+str(index)+'/hvmagent.csv'
    df = pd.read_csv(log_path) #biased data
    lines = []
    total_data_count = 0
    valid_data_count = 0
    logged_class_list = []
    correct_class_list = []
    current_busy_cpu_list = []
    for i in range(len(df.iteration)-1):
        correct_class = df.cpu_max[i+1]
        pred_class = df.pred_peak[i]
        upper_bound = df.upper_bound[i]
        current_busy_cpu = df.primary_busy_cores[i]
        total_data_count += 1
        if upper_bound > correct_class or upper_bound==10:
            # overprediction
            logged_class_list.append(max(pred_class, current_busy_cpu+1))
            correct_class_list.append(correct_class)
            current_busy_cpu_list.append(current_busy_cpu)

            valid_data_count += 1
            features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                        +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                        +" med:"+str(df.f_med[i])
            #lines.append(features)
            '''
            for k in range(1,11):
                if k<correct_class:
                    cost = correct_class-k+10
                else:
                    cost = k-correct_class
                label = str(k)+":"+str(cost)+":1 "
                sample = label+features
                lines.append(sample)
            '''
            k = correct_class
            if k<correct_class:
                cost = correct_class-k+10
            else:
                cost = k-correct_class
            label = str(k)+":"+str(cost)+":1 "
            sample = label+features
            lines.append(sample)

    print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_data.txt", "w")
    for line in lines:
      # write line to output file
      outF.write(line)
      outF.write("\n")
    outF.close()



total data: 11791
total valid data: 10362
total data: 11789
total valid data: 10455
total data: 11789
total valid data: 10518


In [328]:
for index in range(3):
    # evaluate m_partial on biased data
    file = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_pred_prob.txt", 'r') 
    pred_prob_lines = file.readlines() 
    pred_class_list = []
    upper_bound_list = []
    for i in range(len(pred_prob_lines)):
        line = pred_prob_lines[i]
        l = line.strip().split(' ')
        for j in range(10):
            if float(l[j])==1:
                pred=j+1
        upper_bound = pred
        if i==0:
            current_busy_cpu = current_busy_cpu_list[i]
        else:
            current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
            #current_busy_cpu = current_busy_cpu_list[i]
        upper_bound = max(pred, current_busy_cpu+1)
        pred_class_list.append(pred)
        upper_bound_list.append(upper_bound)
        #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_pred.txt", "w")
    for line in pred_class_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_upper_bound.txt", "w")
    for line in upper_bound_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

In [329]:
# extract biased data from explore
dir_name = "results-0524T1732-learning-1-5ms-deploy-full-explore-3iter"
for index in range(3):
    log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/'+str(index)+'/hvmagent.csv'
    df = pd.read_csv(log_path) #biased data
    lines = []
    total_data_count = 0
    valid_data_count = 0
    logged_class_list = []
    correct_class_list = []
    current_busy_cpu_list = []
    for i in range(len(df.iteration)-1):
        correct_class = df.cpu_max[i+1]
        pred_class = df.pred_peak[i]
        upper_bound = df.upper_bound[i]
        current_busy_cpu = df.primary_busy_cores[i]
        total_data_count += 1
        if upper_bound > correct_class or upper_bound==10:
            # overprediction
            logged_class_list.append(max(pred_class, current_busy_cpu+1))
            correct_class_list.append(correct_class)
            current_busy_cpu_list.append(current_busy_cpu)

            valid_data_count += 1
            features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                        +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                        +" med:"+str(df.f_med[i])
            #lines.append(features)
            '''
            for k in range(1,11):
                if k<correct_class:
                    cost = correct_class-k+10
                else:
                    cost = k-correct_class
                label = str(k)+":"+str(cost)+":1 "
                sample = label+features
                lines.append(sample)
            '''
            k = correct_class
            if k<correct_class:
                cost = correct_class-k+10
            else:
                cost = k-correct_class
            label = str(k)+":"+str(cost)+":1 "
            sample = label+features
            lines.append(sample)

    print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_data.txt", "w")
    for line in lines:
      # write line to output file
      outF.write(line)
      outF.write("\n")
    outF.close()

    # evaluate m_partial on biased data
    file = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_pred_prob.txt", 'r') 
    pred_prob_lines = file.readlines() 
    pred_class_list = []
    upper_bound_list = []
    for i in range(len(pred_prob_lines)):
        line = pred_prob_lines[i]
        l = line.strip().split(' ')
        for j in range(10):
            if float(l[j])==1:
                pred=j+1
        upper_bound = pred
        if i==0:
            current_busy_cpu = current_busy_cpu_list[i]
        else:
            current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
            #current_busy_cpu = current_busy_cpu_list[i]
        upper_bound = max(pred, current_busy_cpu+1)
        pred_class_list.append(pred)
        upper_bound_list.append(upper_bound)
        #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_pred.txt", "w")
    for line in pred_class_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

    outF = open("160k/m_partial_biased_data_explore_3iter/"+str(index)+"/160k_upper_bound.txt", "w")
    for line in upper_bound_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

    print len(pred_class_list) # predictions from partial-feedback model
    print len(correct_class_list) # correct labels (ground truth)
    print len(logged_class_list) # correct labels (ground truth)
    assert len(pred_class_list) == len(correct_class_list)
    assert len(pred_class_list) == len(logged_class_list)
    
    print "===Evaluating M_partial on biased data ==="
    no_harvest = True

    print "\n*basic estimator*"
    print "pred_class_list"
    basic_est = basic_estimator(pred_class_list, correct_class_list, no_harvest)
    print "upper_bound_list"
    basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

    print "\n*ips+ estimator*"
    print "pred_class_list"
    ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, True)
    print "upper_bound_list"
    ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, True)

    print ""
    print "pred_class_list"
    ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, False)
    print "upper_bound_list"
    ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, False)

total data: 11791
total valid data: 10362
10362
10362
10362
===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 10362/10362
sum/count = 121079/10362=11.6849063887
upper_bound_list
total valid data/total data: 10362/10362
sum/count = 102954/10362=9.93572669369

*ips+ estimator*
pred_class_list
sum/count = 121127.0/10362=11.6895386991
sum/weights = 121127.0/10366.0=11.6850279761
p=1 for 10361/10362
upper_bound_list
sum/count = 103002.0/10362=9.94035900405
sum/weights = 103002.0/10366.0=9.93652324908
p=1 for 10361/10362

pred_class_list
sum/count = 605395.0/10362=58.4245319436
sum/weights = 605395.0/51810.0=11.6849063887
p=1 for 0/10362
upper_bound_list
sum/count = 514242.0/10362=49.6276780544
sum/weights = 514242.0/51298.0=10.024601349
p=1 for 128/10362
total data: 11789
total valid data: 10455
10455
10455
10455
===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 10455/10455
sum/

In [314]:
print "===Evaluating M_partial on biased data ==="
no_harvest = True

print "\n*basic estimator*"
print "pred_class_list"
basic_est = basic_estimator(pred_class_list, correct_class_list, no_harvest)
print "upper_bound_list"
basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

print "\n*ips+ estimator*"
print "pred_class_list"
ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, True)
print "upper_bound_list"
ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, True)

print ""
print "pred_class_list"
ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, False)
print "upper_bound_list"
ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, False)

===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 10385/10385
sum/count = 120942/10385=11.6458353394
upper_bound_list
total valid data/total data: 10385/10385
sum/count = 103061/10385=9.92402503611

*ips+ estimator*
pred_class_list
sum/count = 120986.0/10385=11.6500722195
sum/weights = 120986.0/10389.0=11.6455866782
p=1 for 10384/10385
upper_bound_list
sum/count = 103105.0/10385=9.92826191623
sum/weights = 103105.0/10389.0=9.92443931081
p=1 for 10384/10385

pred_class_list
sum/count = 604710.0/10385=58.2291766972
sum/weights = 604710.0/51925.0=11.6458353394
p=1 for 0/10385
upper_bound_list
sum/count = 514713.0/10385=49.5631198844
sum/weights = 514713.0/51349.0=10.0238174064
p=1 for 144/10385


In [246]:
def ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, a):
    total_classes = 10
    count = 0
    weights = 0
    sum_ = 0
    n = total_data_count = len(pred_class_list)
    count = 0
    count_p1 = 0
    for i in range(n):
        count += 1;
        action_m = pred_class = pred_class_list[i]
        correct_class = correct_class_list[i]
        if pred_class == correct_class and pred_class != total_classes and not no_harvest:
            # correct_class unknown
            continue 
        if pred_class<correct_class:
            cost = correct_class-pred_class+total_classes
        else:
            cost = pred_class-correct_class
        #---- compute p ----
        full_info_action = correct_class
        if a:
            a_logging_model = logged_class_list[i]
            if a_logging_model > full_info_action:
                count_p1 += 1
                p = 1
                #print "count (a_logging_model {} > full_info_action {}) is {}".format(a_logging_model,full_info_action,count_p1)
            else:
                p = 0.2
        else:
            if action_m > full_info_action:
                count_p1 += 1
                p = 1
                #print "count (action_m {} > full_info_action {}) is {}".format(action_m,full_info_action,count_p1)
            else:
                p = 0.2

        #---- ---- ---- ----
        sum_ += cost/p
        weights += 1/p
        #print "[pred,target,cost]: [{},{},{}]".format(pred_class, correct_class, cost)
    print "sum/count = {}/{}={}".format(sum_, count, sum_/count)
    print "sum/weights = {}/{}={}".format(sum_, weights, sum_/weights)
    print "p=1 for {}/{}".format(count_p1, n)
        

In [306]:
# extract biased data
dir_name = "results-0522T0733-learning-1-5ms-deploy-full"
log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/0/hvmagent.csv'
df = pd.read_csv(log_path) #biased data
lines = []
total_data_count = 0
valid_data_count = 0
logged_class_list = []
correct_class_list = []
current_busy_cpu_list = []
for i in range(len(df.iteration)-1):
    correct_class = df.cpu_max[i+1]
    pred_class = df.pred_peak[i]
    upper_bound = df.upper_bound[i]
    current_busy_cpu = df.primary_busy_cores[i]
    total_data_count += 1
    if upper_bound > correct_class or upper_bound==10:
        # overprediction
        logged_class_list.append(max(pred_class, current_busy_cpu+1))
        correct_class_list.append(correct_class)
        current_busy_cpu_list.append(current_busy_cpu)
        
        valid_data_count += 1
        features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                    +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                    +" med:"+str(df.f_med[i])
        #lines.append(features)
        '''
        for k in range(1,11):
            if k<correct_class:
                cost = correct_class-k+10
            else:
                cost = k-correct_class
            label = str(k)+":"+str(cost)+":1 "
            sample = label+features
            lines.append(sample)
        '''
        k = correct_class
        if k<correct_class:
            cost = correct_class-k+10
        else:
            cost = k-correct_class
        label = str(k)+":"+str(cost)+":1 "
        sample = label+features
        lines.append(sample)

print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)

outF = open("160k/m_partial_biased_data/160k_data.txt", "w")
for line in lines:
  # write line to output file
  outF.write(line)
  outF.write("\n")
outF.close()


total data: 11856
total valid data: 9423


In [315]:
# evaluate m_partial on biased data
file = open('160k/m_partial_biased_data/160k_pred_prob.txt', 'r') 
pred_prob_lines = file.readlines() 
pred_class_list = []
upper_bound_list = []
for i in range(len(pred_prob_lines)):
    line = pred_prob_lines[i]
    l = line.strip().split(' ')
    for j in range(10):
        if float(l[j])==1:
            pred=j+1
    upper_bound = pred
    if i==0:
        current_busy_cpu = current_busy_cpu_list[i]
    else:
        current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
        #current_busy_cpu = current_busy_cpu_list[i]
    upper_bound = max(pred, current_busy_cpu+1)
    pred_class_list.append(pred)
    upper_bound_list.append(upper_bound)
    #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])
    
outF = open("160k/m_partial_biased_data/160k_pred.txt", "w")
for line in pred_class_list:
    # write line to output file
    #print line
    outF.write(str(line))
    outF.write("\n")
outF.close()

outF = open("160k/m_partial_biased_data/160k_upper_bound.txt", "w")
for line in upper_bound_list:
    # write line to output file
    #print line
    outF.write(str(line))
    outF.write("\n")
outF.close()

In [317]:
print "===Evaluating M_partial on biased data ==="
no_harvest = True

print "\n*basic estimator*"
print "pred_class_list"
basic_est = basic_estimator(pred_class_list, correct_class_list, no_harvest)
print "upper_bound_list"
basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

print "\n*ips+ estimator*"
print "pred_class_list"
ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, True)
print "upper_bound_list"
ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, True)

print ""
print "pred_class_list"
ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, False)
print "upper_bound_list"
ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, False)

===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 9423/9423
sum/count = 109805/9423=11.6528706357
upper_bound_list
total valid data/total data: 9423/9423
sum/count = 93626/9423=9.93590151756

*ips+ estimator*
pred_class_list
sum/count = 109849.0/9423=11.6575400616
sum/weights = 109849.0/9427.0=11.6525936141
p=1 for 9422/9423
upper_bound_list
sum/count = 93670.0/9423=9.94057094344
sum/weights = 93670.0/9427.0=9.93635302854
p=1 for 9422/9423

pred_class_list
sum/count = 549025.0/9423=58.2643531784
sum/weights = 549025.0/47115.0=11.6528706357
p=1 for 0/9423
upper_bound_list
sum/count = 467622.0/9423=49.6255969436
sum/weights = 467622.0/46619.0=10.0307170896
p=1 for 124/9423


In [330]:
# extract biased data from explore
dir_name = "results-0524T1743-learning-1-5ms-deploy-full-3iter"
for index in range(3):
    log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/'+str(index)+'/hvmagent.csv'
    df = pd.read_csv(log_path) #biased data
    lines = []
    total_data_count = 0
    valid_data_count = 0
    logged_class_list = []
    correct_class_list = []
    current_busy_cpu_list = []
    for i in range(len(df.iteration)-1):
        correct_class = df.cpu_max[i+1]
        pred_class = df.pred_peak[i]
        upper_bound = df.upper_bound[i]
        current_busy_cpu = df.primary_busy_cores[i]
        total_data_count += 1
        if upper_bound > correct_class or upper_bound==10:
            # overprediction
            logged_class_list.append(max(pred_class, current_busy_cpu+1))
            correct_class_list.append(correct_class)
            current_busy_cpu_list.append(current_busy_cpu)

            valid_data_count += 1
            features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                        +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                        +" med:"+str(df.f_med[i])
            #lines.append(features)
            '''
            for k in range(1,11):
                if k<correct_class:
                    cost = correct_class-k+10
                else:
                    cost = k-correct_class
                label = str(k)+":"+str(cost)+":1 "
                sample = label+features
                lines.append(sample)
            '''
            k = correct_class
            if k<correct_class:
                cost = correct_class-k+10
            else:
                cost = k-correct_class
            label = str(k)+":"+str(cost)+":1 "
            sample = label+features
            lines.append(sample)

    print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)

    outF = open("160k/m_partial_biased_data_3iter/"+str(index)+"/160k_data.txt", "w")
    for line in lines:
      # write line to output file
      outF.write(line)
      outF.write("\n")
    outF.close()


total data: 11856
total valid data: 9849
total data: 11854
total valid data: 9719
total data: 11852
total valid data: 9825


In [335]:
# extract biased data
dir_name = "results-0524T1743-learning-1-5ms-deploy-full-3iter"
for index in range(3):
    log_path = '160k/logs/'+dir_name+'/SmartIPI_HOLES-0.1-sleep_ms/buf-1/'+str(index)+'/hvmagent.csv'
    df = pd.read_csv(log_path) #biased data
    lines = []
    total_data_count = 0
    valid_data_count = 0
    logged_class_list = []
    correct_class_list = []
    current_busy_cpu_list = []
    for i in range(len(df.iteration)-1):
        correct_class = df.cpu_max[i+1]
        pred_class = df.pred_peak[i]
        upper_bound = df.upper_bound[i]
        current_busy_cpu = df.primary_busy_cores[i]
        total_data_count += 1
        if upper_bound > correct_class or upper_bound==10:
            # overprediction
            logged_class_list.append(max(pred_class, current_busy_cpu+1))
            correct_class_list.append(correct_class)
            current_busy_cpu_list.append(current_busy_cpu)

            valid_data_count += 1
            features = "|busy_cores_prev_interval min:"+str(df.f_min[i])+" max:"+str(df.f_max[i]) \
                        +" avg:"+str(df.f_avg[i])+" stddev:"+str(df.f_stddev[i]) \
                        +" med:"+str(df.f_med[i])
            #lines.append(features)
            '''
            for k in range(1,11):
                if k<correct_class:
                    cost = correct_class-k+10
                else:
                    cost = k-correct_class
                label = str(k)+":"+str(cost)+":1 "
                sample = label+features
                lines.append(sample)
            '''
            k = correct_class
            if k<correct_class:
                cost = correct_class-k+10
            else:
                cost = k-correct_class
            label = str(k)+":"+str(cost)+":1 "
            sample = label+features
            lines.append(sample)

    print "total data: {}\ntotal valid data: {}".format(total_data_count, valid_data_count)

    outF = open("160k/m_partial_biased_data_3iter/"+str(index)+"/160k_data.txt", "w")
    for line in lines:
      # write line to output file
      outF.write(line)
      outF.write("\n")
    outF.close()

    # evaluate m_partial on biased data
    file = open("160k/m_partial_biased_data_3iter/"+str(index)+"/160k_pred_prob.txt", 'r') 
    pred_prob_lines = file.readlines() 
    pred_class_list = []
    upper_bound_list = []
    for i in range(len(pred_prob_lines)):
        line = pred_prob_lines[i]
        l = line.strip().split(' ')
        for j in range(10):
            if float(l[j])==1:
                pred=j+1
        upper_bound = pred
        if i==0:
            current_busy_cpu = current_busy_cpu_list[i]
        else:
            current_busy_cpu = min(current_busy_cpu_list[i], upper_bound_list[i-1])
            #current_busy_cpu = current_busy_cpu_list[i]
        upper_bound = max(pred, current_busy_cpu+1)
        pred_class_list.append(pred)
        upper_bound_list.append(upper_bound)
        #print "pred_class={}\tupper_bound={}\tcurrent_busy={}\tcorrect_class{}".format(pred, upper_bound,current_busy_cpu,correct_class_list[i])

    outF = open("160k/m_partial_biased_data_3iter/"+str(index)+"/160k_pred.txt", "w")
    for line in pred_class_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

    outF = open("160k/m_partial_biased_data_3iter/"+str(index)+"/160k_upper_bound.txt", "w")
    for line in upper_bound_list:
        # write line to output file
        #print line
        outF.write(str(line))
        outF.write("\n")
    outF.close()

    print len(pred_class_list) # predictions from partial-feedback model
    print len(correct_class_list) # correct labels (ground truth)
    print len(logged_class_list) # correct labels (ground truth)
    assert len(pred_class_list) == len(correct_class_list)
    assert len(pred_class_list) == len(logged_class_list)
    
    print "===Evaluating M_partial on biased data ==="
    no_harvest = True

    print "\n*basic estimator*"
    print "pred_class_list"
    basic_est = basic_estimator(pred_class_list, correct_class_list, no_harvest)
    print "upper_bound_list"
    basic_est = basic_estimator(upper_bound_list, correct_class_list, no_harvest)

    print "\n*ips+ estimator*"
    print "pred_class_list"
    ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, True)
    print "upper_bound_list"
    ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, True)

    print ""
    print "pred_class_list"
    ips_plus_est = ips_plus_estimator(pred_class_list, correct_class_list, logged_class_list, no_harvest, False)
    print "upper_bound_list"
    ips_plus_est = ips_plus_estimator(upper_bound_list, correct_class_list, logged_class_list, no_harvest, False)

total data: 11856
total valid data: 9849
9849
9849
9849
===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 9849/9849
sum/count = 117079/9849=11.887399736
upper_bound_list
total valid data/total data: 9849/9849
sum/count = 102798/9849=10.4374048127

*ips+ estimator*
pred_class_list
sum/count = 117127.0/9849=11.8922733272
sum/weights = 117127.0/9853.0=11.8874454481
p=1 for 9848/9849
upper_bound_list
sum/count = 102798.0/9849=10.4374048127
sum/weights = 102798.0/9853.0=10.4331675632
p=1 for 9848/9849

pred_class_list
sum/count = 585395.0/9849=59.4369986801
sum/weights = 585395.0/49245.0=11.887399736
p=1 for 0/9849
upper_bound_list
sum/count = 513654.0/9849=52.1529089248
sum/weights = 513654.0/48917.0=10.5005212912
p=1 for 82/9849
total data: 11854
total valid data: 9719
9719
9719
9719
===Evaluating M_partial on biased data ===

*basic estimator*
pred_class_list
total valid data/total data: 9719/9719
sum/count = 115587/9719=11.8928902